In [12]:
import torch
from vit import ViT

# model1 = ResNet50ViT(img_dim=128, pretrained_resnet=False, 
#                         blocks=6, num_classes=10, 
#                         dim_linear_block=256, dim=256)
# # or
model = ViT(img_dim=256, in_channels=3, patch_dim=16, num_classes=10,dim=512)
x = torch.rand(2, 3, 256, 256)
y = model(x)
print(y.shape)

torch.Size([2, 10])


# Experiment with TransOCR from 
https://github.com/FudanVI/benchmarking-chinese-text-recognition/blob/main/models/TransOCR/model/transocr.py


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from transocr import Transformer
from utils import get_data_package, converter, tensor2str, get_alphabet
import zhconv

# Define your variables
exp_name = 'test'
batch_size = 32
lr = 1.0
epoch = 1000
radical = False
test = False
resume = ''
train_dataset = ''
test_dataset = ''
imageH = 32
imageW = 256
coeff = 1.0
alpha_path = './data/benchmark.txt'
alpha_path_radical = './data/radicals.txt'
decompose_path = './data/decompose.txt'

alphabet = get_alphabet({'radical': radical}, 'char')  # Modify this as needed
print('alphabet:', alphabet)

model = Transformer({'imageH': imageH, 'imageW': imageW}).cuda()
model = nn.DataParallel(model)
train_loader, test_loader = get_data_package({'batch_size': batch_size, 'train_dataset': train_dataset, 'test_dataset': test_dataset})
optimizer = optim.Adadelta(model.parameters(), lr=lr, rho=0.9, weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss().cuda()
best_acc = -1

if resume.strip() != '':
    model.load_state_dict(torch.load(resume))
    print('loading pretrained model！！！')

def train(epoch, iteration, image, length, text_input, text_gt, length_radical, radical_input, radical_gt):
    model.train()
    optimizer.zero_grad()
    result = model(image, length, text_input, length_radical, radical_input)

    text_pred = result['pred']
    loss_char = criterion(text_pred, text_gt)
    if radical:
        radical_pred = result['radical_pred']
        loss_radical = criterion(radical_pred, radical_gt)
        loss = loss_char + coeff * loss_radical
        print(
            'epoch : {} | iter : {}/{} | loss : {} | char : {} | radical : {} '.format(epoch, iteration, len(train_loader), loss, loss_char, loss_radical))
    else:
        loss = loss_char
        print('epoch : {} | iter : {}/{} | loss : {}'.format(epoch, iteration, len(train_loader), loss))
    loss.backward()
    optimizer.step()

test_time = 0
@torch.no_grad()
def test(epoch):
    torch.cuda.empty_cache()
